# QCCS: PQSC-HDAWG-UHFQA feedforward experiment for active qubit reset

###### credit to Niels and Hassan for some of material used

### Table of Contents

* [1) Imports and helper functions](#chapter1)
* [2) Device initialization/connection](#chapter2)
* [3) Configure PQSC](#chapter3)
* [4) Configure PQSC](#chapter4)
* [5) Configure UHFQA](#chapter5)
    * [5.2 Configure UHFQA QCCS mode](#section_5_1)
    * [5.2 Configure UHFQA for 2-qubit](#section_5_2)
* [6) Configure HDAWG](#chapter6)
* [7) Configure Upload and Run program](#chapter7)
* [8) Read results (under work)](#chapter8)

### 1) Imports and helper functions <a class="anchor" id="chapter1"></a>

In [1]:
import time
import zhinst.ziPython as zi
import numpy as np
import math

# Compile AWG seqC code
def awg_compile_upload_elf(awgModule, awgIndex, awg_program):
    """Compile and upload awg_program as .elf file"""
    awgModule.set('awgModule/index', awgIndex)
    awgModule.set('awgModule/compiler/sourcestring', awg_program)

    while awgModule.getInt('awgModule/compiler/status') == -1:
        time.sleep(0.1)
    if awgModule.getInt('awgModule/compiler/status') == 1:
        raise Exception(awgModule.getString('awgModule/compiler/statusstring'))

    if awgModule.getInt('awgModule/compiler/status') == 2:
        print("Compilation successful with warnings, will upload the program to the instrument.")
        print("Compiler warning: ", awgModule.getString('awgModule/compiler/statusstring'))

    time.sleep(0.2)
    i = 0
    while (awgModule.getDouble('awgModule/progress') < 1.0) and (awgModule.getInt('awgModule/elf/status') != 1):
        time.sleep(0.5)
        i += 1

    if awgModule.getInt('awgModule/elf/status') == 1:
        raise Exception("Upload to the instrument failed.")

### 2) Specify your data server + devices used <a class="anchor" id="chapter2"></a>
#### Note: variable UHFQA_watch can be ignored

In [2]:
SERVER = '127.0.0.1'

daq = zi.ziDAQServer(SERVER, 8004, 1)

# Define device IDs
PQSC = 'dev10006'
UHFQA = 'dev2171'
HDAWG = 'dev8147'

# On the continuity setup, this device is not connected to anything or perform any function
UHFQA_watch = 'dev2004' 


# Connection to data server
daq.connectDevice(PQSC, '1gbe')
daq.connectDevice(UHFQA, '1gbe')
daq.connectDevice(HDAWG, '1gbe')

### 3) Configure the PQSC for register forwarding (conditional unit TBD)  <a class="anchor" id="chapter3"></a>

In [3]:
### Configure PQSC

# Use external reference clock coming from the UHFQA
daq.setInt(f'/{PQSC}/system/clocks/referenceclock/in/source', 1)

## Configure execution engine
# Send a single trigger to start the HDAWG and the UHFQA
daq.setInt(f'/{PQSC}/execution/repetitions', 1)
# Wait for feedback to arrive from UHFQA (10us is more than enough)
daq.setDouble(f'/{PQSC}/execution/holdoff', 10e-6)

## Register forwarding
# ZSync output port to the receiver HDAWG
port = 0
# Program register bank forwarding
fwd_length = 4
fwd = range(0, 4)
daq.setVector(f'/{PQSC}/raw/regs/{port}/fwd', np.array(fwd).astype(np.uint32))
# Enable forwarding on output port
daq.setInt(f'/{PQSC}/raw/zsyncs/{port}/txmux/fwd_en', 1)

## Condition unit
# ZSync output port to the receiver HDAWG
port = 0
# Program condition unit sources
src_length = 8
src = range(0, 8)
daq.setVector(f'/{PQSC}/raw/cond/src', np.array(src).astype(np.uint32))
# Program condition unit function
lut_length = 2**src_length
offset = 512
lut = range(offset, offset+lut_length)
daq.setVector(f'/{PQSC}/raw/cond/lut', np.array(lut).astype(np.uint32))
# Program condition unit byte selection
sel_length = 18
sel = [1] * sel_length
daq.setVector(f'/{PQSC}/raw/cond/sel', np.array(sel).astype(np.uint32))

# Enable condition output on forwarding port
daq.setInt(f'/{PQSC}/raw/zsyncs/{port}/txmux/cond_en', 0) # This should be 0 according to Niels, cond not working

### 4) Configure HDAWG to QCCS mode  <a class="anchor" id="chapter4"></a>

In [4]:
## Configure HDAWG

# Use ZSync clock
daq.setInt(f'/{HDAWG}/system/clocks/referenceclock/source', 2)

## Configure DIO
# Configure DIO switch to QCCS mode
daq.setInt(f'/{HDAWG}/dios/0/mode', 3)
# Drive the two most significant bytes of the DIO port
# The UHFQA drives the two least significant bytes of the DIO port
daq.setInt(f'/{HDAWG}/dios/0/drive', 0b1100)
# Configure DIO triggering to match ZSync input
daq.setInt(f'{HDAWG}/awgs/0/dio/strobe/slope', 0)
daq.setInt(f'{HDAWG}/awgs/0/dio/valid/polarity', 0)

## Configure AWG
# Setup AWG module
awg_hd = daq.awgModule()
awg_hd.set('device', HDAWG)
awg_hd.set('index', 0)
awg_hd.execute()

# Turn on HDAWG outputs
daq.setInt(f'/{HDAWG}/sigouts/0/on', 1)
daq.setInt(f'/{HDAWG}/sigouts/1/on', 1)

### 5) Configure UHFQA <a class="anchor" id="chapter5"></a>

### 5.1) Configure UHFQA for QCCS mode (underwork) <a class="anchor" id="section_5_1"></a>

In [5]:
### Configure UHFQA

# UHFQA uses its own clock in this experiment

## Configure DIO
# Sample DIO data at 50 MHz
daq.setInt(f'{UHFQA}/dios/0/extclk', 2)
# Set DIO output to QA result QCCS
daq.setInt(f'{UHFQA}/dios/0/mode', 4)
# Drive the two least significant bytes of the DIO port
# The HDAWG drives the two most significant bytes of the DIO port
daq.setInt(f'{UHFQA}/dios/0/drive', 0b0011)
# Configure DIO triggering to match HDAWG DIO input
daq.setInt(f'/{UHFQA}/awgs/0/dio/strobe/slope', 0)
daq.setInt(f'/{UHFQA}/awgs/0/dio/valid/polarity', 2)
daq.setInt(f'/{UHFQA}/awgs/0/dio/valid/index', 16)

## QA readout configuration
# Bypass crosstalk to reduce latency
daq.setInt(f'{UHFQA}/qas/0/crosstalk/bypass', 1)
# Set threshold levels to a low value so qubit readout is always high
for i in range(10):
    daq.setDouble(f'/{UHFQA}/qas/0/thresholds/{i}/level', -100)
# Reset QA results
daq.setInt(f'{UHFQA}/qas/0/result/reset', 1)

### 5.2) Configure UHFQA for two-qubit readout + upload integration weights <a class="anchor" id="section_5_2"></a>

In [6]:
# Input/Output settings
daq.setInt(f'/{UHFQA}/sigins/0/imp50', 1)
daq.setInt(f'/{UHFQA}/sigins/1/imp50', 1)
daq.setDouble(f'/{UHFQA}/sigins/0/range', 1.5)
daq.setDouble(f'/{UHFQA}/sigins/1/range', 1.5)
daq.setInt(f'/{UHFQA}/sigouts/0/on', 1)
daq.setInt(f'/{UHFQA}/sigouts/1/on', 1)


# Configure QA setup
# 128 is the length of the readout I/Q signal read by the QA
daq.setDouble(f'/{UHFQA}/qas/0/integration/length', 128)
daq.setInt(f'/{UHFQA}/qas/0/integration/mode', 0)
daq.setInt(f'/{UHFQA}/qas/0/integration/sources/0', 0)
daq.setInt(f'/{UHFQA}/qas/0/integration/sources/1', 0)


# Upload integration weights
ch1_freq = 28.125e6 # Oscillator frequency
ch2_freq = 56.25e6 # Oscillator frequency
fs_uhfqa = 1.8e9 # Sampling frequency of UHF-QA
integrationPoints = 4096 # Number of sampled used for integration
ampl = 0.38 # Vpk of weights signal

ch1_w_real = ampl*np.sin(2*np.pi*ch1_freq*np.arange(integrationPoints)/fs_uhfqa)
ch1_w_imag = ampl*np.cos(2*np.pi*ch1_freq*np.arange(integrationPoints)/fs_uhfqa)

ch2_w_real = ampl*np.sin(2*np.pi*ch2_freq*np.arange(integrationPoints)/fs_uhfqa)
ch2_w_imag = ampl*np.cos(2*np.pi*ch2_freq*np.arange(integrationPoints)/fs_uhfqa)

daq.setVector(f'/{UHFQA}/qas/0/integration/weights/0/real', ch1_w_real)
daq.setVector(f'/{UHFQA}/qas/0/integration/weights/0/imag', ch1_w_imag)

daq.setVector(f'/{UHFQA}/qas/0/integration/weights/1/real', ch2_w_real)
daq.setVector(f'/{UHFQA}/qas/0/integration/weights/1/imag', ch2_w_imag)


#### State discrimination parameters
## Recommended to run the calibration scripts first to find the delays and best parameters before choosing

sampleDelay = 228

ch0rot = 316.5
ch1rot = 316.5

ch0_threshold = 3.389212368362104
ch1_threshold = 3.173179186089377

# Convert to cartesian co-ordinates
ch0rot = complex(np.cos(math.radians(ch0rot)), np.sin(math.radians(ch0rot)))
ch1rot = complex(np.cos(math.radians(ch1rot)), np.sin(math.radians(ch1rot)))

# Set rotations to maximise values into the real plane
daq.setComplex(f'/{UHFQA}/qas/0/rotations/0', ch0rot)
daq.setComplex(f'/{UHFQA}/qas/0/rotations/1', ch1rot) # JS: -1 + 0j for Elisa's setup, 0.707 - 0.707j for Continuity setup

# Set the optimum thresholds for channels 0 and 1
daq.setDouble(f'/{UHFQA}/qas/0/thresholds/0/level', ch0_threshold)
daq.setDouble(f'/{UHFQA}/qas/0/thresholds/1/level', ch1_threshold)

daq.setInt(f'/{UHFQA}/qas/0/delay', sampleDelay) # JS: Need to adjust this accordingly to whatever setup we use. 276 is what I found optimal on Continuity setup 7/13/2020. 268 is for my setup

### 6) Compile & upload AWG programs and start the program for four repetitions  <a class="anchor" id="chapter6"></a>

In [7]:
UHF_qubitSim_prog = '''
const ampl = 0.5;
 
wave I_q0_state_0 = zeros(128);
wave Q_q0_state_0 = zeros(128);
wave I_q0_state_1 = sine(128, ampl, 0, 2);
wave Q_q0_state_1 = cosine(128, ampl, 0, 2);
 
wave I_q1_state_0 = zeros(128);
wave Q_q1_state_0 = zeros(128);
wave I_q1_state_1 = sine(128, ampl, 0, 4);
wave Q_q1_state_1 = cosine(128, ampl, 0, 4);
 
wave I_state_00 = I_q1_state_0 + I_q0_state_0;
wave Q_state_00 = Q_q1_state_0 + Q_q0_state_0;
 
wave I_state_01 = I_q1_state_0 + I_q0_state_1;
wave Q_state_01 = Q_q1_state_0 + Q_q0_state_1;
 
wave I_state_10 = I_q1_state_1 + I_q0_state_0;
wave Q_state_10 = Q_q1_state_1 + Q_q0_state_0;
 
wave I_state_11 = I_q1_state_1 + I_q0_state_1;
wave Q_state_11 = Q_q1_state_1 + Q_q0_state_1;

var qubitState = getUserReg(0); 
var qubitState2 = getUserReg(1);
var qubitState3 = getUserReg(2); 
var qubitState4 = getUserReg(3);

waitDIOTrigger();
setID(0);


//repeat(4){
switch (qubitState) {
  case 0: playWave(I_state_00, Q_state_00);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 1: playWave(I_state_01, Q_state_01);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 2: playWave(I_state_10, Q_state_10);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 3: playWave(I_state_11, Q_state_11);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
}

waitDIOTrigger();
setID(0);
switch (qubitState2) {
  case 0: playWave(I_state_00, Q_state_00);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 1: playWave(I_state_01, Q_state_01);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 2: playWave(I_state_10, Q_state_10);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 3: playWave(I_state_11, Q_state_11);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
}

waitDIOTrigger();
setID(0);
switch (qubitState3) {
  case 0: playWave(I_state_00, Q_state_00);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 1: playWave(I_state_01, Q_state_01);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 2: playWave(I_state_10, Q_state_10);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 3: playWave(I_state_11, Q_state_11);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
}

waitDIOTrigger();
setID(0);
switch (qubitState4) {
  case 0: playWave(I_state_00, Q_state_00);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 1: playWave(I_state_01, Q_state_01);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 2: playWave(I_state_10, Q_state_10);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
  case 3: playWave(I_state_11, Q_state_11);startQAResult(0x300 << 16, 0b1111);waitQAResultTrigger();
}




'''

HD_rstpulse_prog = '''
setUserReg(0,0);
setUserReg(1,0);
setUserReg(2,0);
setUserReg(3,0);

wave w = gauss(48, 1, 24, 16);
//while(1){
 // playWave(w, -w); 
//}

// At start of program
waitDIOTrigger();
// Wait for qubit measurement result
waitDIOTrigger();
var res = getDIOTriggered();
switch (res) {
  case 3072: playWave(w, w); 
  case 3328: playWave(w, -w);
  case 3584: playWave(-w, w);
  case 3840: playWave(-w, -w);
}

waitDIOTrigger();
// Wait for qubit measurement result
waitDIOTrigger();
var res2 = getDIOTriggered();
switch (res2) {
  case 3072: playWave(w, w); 
  case 3328: playWave(w, -w);
  case 3584: playWave(-w, w);
  case 3840: playWave(-w, -w);
}

waitDIOTrigger();
// Wait for qubit measurement result
waitDIOTrigger();
var res3 = getDIOTriggered();
switch (res3) {
  case 3072: playWave(w, w); 
  case 3328: playWave(w, -w);
  case 3584: playWave(-w, w);
  case 3840: playWave(-w, -w);
}

waitDIOTrigger();
// Wait for qubit measurement result
waitDIOTrigger();
var res4 = getDIOTriggered();
switch (res4) {
  case 3072: playWave(w, w); 
  case 3328: playWave(w, -w);
  case 3584: playWave(-w, w);
  case 3840: playWave(-w, -w);
}



setUserReg(0, res);
setUserReg(1, res2);
setUserReg(2, res3);
setUserReg(3, res4);
'''

# Qubit state 11, 10, 01, 00
daq.setInt(f'/{UHFQA}/awgs/0/userregs/0', 3)
daq.setInt(f'/{UHFQA}/awgs/0/userregs/1', 2)
daq.setInt(f'/{UHFQA}/awgs/0/userregs/2', 1)
daq.setInt(f'/{UHFQA}/awgs/0/userregs/3', 0)



# Setup UHF AWG module
awg_uhf = daq.awgModule()
awg_uhf.set('device', UHFQA)
awg_uhf.set('index', 0)
awg_uhf.execute()
awg_uhf.set('awg/enable', 0)

# Setup HD AWG module
awg_hd = daq.awgModule()
awg_hd.set('device', HDAWG)
awg_hd.set('index', 0)
awg_hd.execute()
awg_hd.set('awg/enable', 0)

# Upload HD program
awg_compile_upload_elf(awg_uhf, 0, UHF_qubitSim_prog)

# Upload UHF program
awg_compile_upload_elf(awg_hd, 0, HD_rstpulse_prog)


# Start the UHF-AWG which will now wait for a DIO trigger from PQSC (via HD)
awg_uhf.set('awg/enable', 1)
#daq.setInt(f'{UHFQA}/awgs/0/enable', 1)
time.sleep(2)
# Start the HD-AWG which will now wait for a DIO trigger from PQSC through direct ZSync connection
awg_hd.set('awg/enable', 1)
#daq.setInt(f'{HDAWG}/awgs/0/enable', 1)
time.sleep(2)


# PQSC holdoff time in seconds (must be multiple of 10)
daq.setDouble(f'/{PQSC}/execution/repetitions', 4)
daq.setDouble(f'/{PQSC}/execution/holdoff', 100e-6)

# Let's begin our measurement now
time.sleep(2)
daq.setInt(f'/{PQSC}/execution/enable', 1)

### 7) Check the four user registers in HDAWG and display feed-forward results (underwork)  <a class="anchor" id="chapter7"></a>

In [11]:
possibleStates = 4

# Read and print out values of HD registers
regs = np.zeros(4)
regs = [daq.getInt(f'/{HDAWG}/awgs/0/userregs/{i}') for i in range(possibleStates)]

hexFormat = "00"
scale = 16 ## equals to hexadecimal
num_of_bits = 8
for i in range(0, possibleStates):
    MSB2 = format(int(hex(regs[i]), 16), '0>42b')[32:34] # Getting the two most significant bits (qubit states) readout
    expectedState = (possibleStates-i)
    print(f"At iteration {i}, the qubit state was {MSB2} aka {regs[i]}")

At iteration 0, the qubit state was 11 aka 3840
At iteration 1, the qubit state was 10 aka 3584
At iteration 2, the qubit state was 01 aka 3328
At iteration 3, the qubit state was 00 aka 3072
